In [75]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter
import networkx as nx

In [76]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


In [77]:
def loadGraph(input_file):
    global d
    G = nx.Graph()
    for j in range(0,n):
        G.add_node(j)

    f = open(input_file, "r")
    string = f.readline()
    string = f.readline()
    string = f.readline()
    for i in range(0, m):
        string = f.readline()
        string = string.split()
        j = int(string[0])-1
        k = int(string[1])-1
        G.add_edge(j, k)
    f.close()
    d = []
    j = 1
    for i in range(n):
        d.append([float("inf")]*(j))
        j+=1
    for i in range(G.number_of_nodes()):
        ds = nx.single_source_shortest_path_length(G, i)
        for key in ds:
            if i >= key:
                d[i][key] = ds[key]
            else:
                d[key][i] = ds[key]
    print("n: " + str(G.number_of_nodes()))
    print("m: " + str(G.number_of_edges()))
    print("ncc: " + str(nx.number_connected_components(G)))

In [78]:

def generate_random_burning_sequence(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa

    current_node = random.choice(list(NB))
    
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            #print("fb",FB)
            #print("nb", NB)
            random_node = random.choice(list(NB))
          
            S.append(random_node)
          
        '''newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  '''
        
        for new in FB:
            B.add(new)
            NB.remove(new)#dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    candidates_for_best.append(node)
            if candidates_for_best:
                best_node = random.choice(candidates_for_best)
            else:
                best_node = None        
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

def generate_multiple_random_burning_sequences(graph, num_sequences):
    
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence(graph)
        if sequence is not None and sequence not in sequences:  
            sequences.append(sequence)
    return sequences


filename = "grafovi/manji/karate.mtx"
graph = load_graph_from_mtx(filename)


#max_burn_nodes = len(graph) // 2
num_sequences = 20

burning_sequences = generate_multiple_random_burning_sequences(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
#for seq in burning_sequences:
 #   print(seq,"len:", len(seq))
print(burning_sequences)  


Generisane burning sekvence:
[[14, 26, 17], [11, 29, 21, 10], [10, 13, 17, 5], [10, 7, 18, 12], [15, 22, 17, 11], [30, 13, 17, 6], [7, 29, 21, 30], [24, 17, 22, 18, 12], [7, 33, 27, 28], [16, 1, 25, 26], [19, 18, 17, 4], [27, 17, 13, 22, 18], [10, 17, 26, 25], [34, 17, 13, 8], [24, 13, 17, 22], [24, 18, 17, 12], [28, 7, 18, 13], [31, 13, 17, 5], [12, 25, 16, 27, 23], [30, 11, 17, 7]]


In [90]:
def transpose_list_of_lists(input_list):
    """
    Transponuje listu listi tako da prvi elementi svih podlisti budu u prvoj listi, drugi u drugoj, itd.
    Takođe, uklanja duplikate iz svake liste u transponovanoj listi.
    """
    transposed = []
    for item in zip(*input_list):
        # Uklanjanje duplikata pretvaranjem u set, a zatim nazad u listu
        unique_item_list = list(set(item))
        transposed.append(unique_item_list)
    return transposed

# Primer sa burning_sequences
burning_sequences_t = transpose_list_of_lists(burning_sequences)
print(burning_sequences_t)


[[34, 7, 10, 11, 12, 14, 15, 16, 19, 24, 27, 28, 30, 31], [33, 1, 7, 11, 13, 17, 18, 22, 25, 26, 29], [13, 16, 17, 18, 21, 22, 25, 26, 27]]


In [94]:
    
def ILP(L,k):
    global d
    
    #model
    model = LpProblem("ILP_COV", LpMinimize)
    #definisanje varijabli
    x = LpVariable.dicts('x', (range(0, k), range(0, n)), cat='Binary')
    b = LpVariable.dict('b',range(0, n), cat='Binary')
    
    #funkcija cilja
    model += lpSum(x[i][j] for i in range(0, k) for j in range(0, n))
    
    #domensko ogranicenje
    
    for i in range(int(k/2)+1):
        print("i:",burning_sequences_t[i])
        for j in burning_sequences_t[i]:
            print("j:",j)
            model += lpSum(x[i][j]) == 1 
    
    #ogranicenja
    #(13)
    for i in range(k):
        suma=0
        for j in range(n):
            suma+=x[i][j]
        model +=(suma<=1)
    #(14)
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        model +=(b[j]<=suma)
    #(12)
    for i in range(k):
        sum2 = 0
        if i == 0:
            sum1 = 1
        else:
            sum1 = 0
        for j in range(n):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        model +=(sum2<=sum1)

    
    #(15)
    model +=lpSum(b[j] for j in range(0, n)) == n 
    
    
    # rjesavanje
    model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0))
  

    if LpStatus[model.status] == 'Optimal':
        print(f"Optimal Solution Found: {model.objective.value()}")
        for i in range(k):
            for j in range(n):
                if x[i][j].value() == 1:
                    print(f"x[{i}, {j}] = 1")
    else:
        print("No optimal solution found")
    s = []
    
    x_out = [[0]*n for i in range(k)]
    for r in range(k):
        
        i = 0
        for e in x_out[r]:
            if e == 1:
                s.append(i)
                break
            i += 1
 
    s.reverse()
    print("OPT: " + str(s))

    return s
        
def main(ni, mi, input_file, Li, Ui):
    n = ni
    m = mi
    U = Ui
    L = Li
    loadGraph(input_file)
    s = ILP(L,U)

if __name__ == "__main__":
    bs = []
    n = 0
    m = 0
    d = {}
    bs_size = float("inf")
    folder_dataset = 'grafovi/manji/'
    dataset = [
        ['karate.mtx',34,78,2,3]
        ]
    for i in range(len(dataset)):
        print("________________________________________________")
        instance = dataset[i][0]
        print("instance: " + instance)
        n = dataset[i][1]
        m = dataset[i][2]
        L = dataset[i][3]
        U = dataset[i][4]
        main(n, m, folder_dataset + dataset[i][0], L, U)
    
    

________________________________________________
instance: karate.mtx
n: 34
m: 78
ncc: 1
i: [34, 7, 10, 11, 12, 14, 15, 16, 19, 24, 27, 28, 30, 31]
j: 34


KeyError: 34